In [1]:
import time
import re
import requests
import yaml
import os.path
from datetime import datetime

In [2]:
prefix='BUNNYFOOFOO'

if os.path.isfile('registry_url.yaml'):
        with open('registry_url.yaml', 'r') as fh:
            regurl = yaml.load(fh)
else:
    print("halp. halp. I've fallen and can't get up")

then = time.time()    
for reg in regurl:
    response = requests.head(reg + prefix)
    if response.status_code == requests.codes.ok:
        print(prefix + ' found in ' + response.url)
        break
interval = time.time() - then
print('checked ' + str(len(regurl)) + ' remote sites for ' + prefix + ' in ' + str(interval)) 

checked 7 remote sites for BUNNYFOOFOO in 5.05529260635376


Quick and dirty explore alternatives to screen scraping
for prefixping service. 

basicly instead of scraping the search results page  
find the yaml file used to generate the page 

for the GO registery the file is: 
[db-xrefs.yaml](http://current.geneontology.org/metadata/db-xrefs.yaml)
for the n3t/identifiers registery the file is:  
[cdl_ebi_prefixes.yaml](https://n2t.net/e/cdl_ebi_prefixes.yaml)

In [3]:
gocurl = 'http://current.geneontology.org/metadata/db-xrefs.yaml'
gocfile = 'db-xrefs.yaml'
gocprefixfile = 'gocprefix.yaml'

then = time.time()
response = requests.head(gocurl)
time.time() - then

then = time.time()
response = requests.get(gocurl)
time.time() - then

`head` is faster than `get` for finding `last_modified`   
but not enough to worry about right now

In [4]:
then = time.time() 
gocprefix = []
response = requests.get(gocurl)
if response.status_code == requests.codes.ok:
    gocxref = yaml.load(response.text)
    remote_last_modified = response.headers['Last-Modified'] 
    # Tue, 01 Aug 2017 14:14:26 GMT   note see http://strftime.org/
    fmt = '%a, %d %b %Y %H:%M:%S GMT'
    remote_datetime = datetime.strptime(remote_last_modified, fmt)
    gocprefix.append('# ' + str(remote_datetime))
    for db in gocxref:
         gocprefix.append(db['database'])
else: 
    print('ERROR ' + response.url + ' returned '+ str(response.status_code))
    print('Trying local cache')
    remote_datetime = None
    if os.path.isfile(gocprefixfile):
        with open(gocprefixfile, 'r') as fh:
            gocprefix = yaml.load(fh)
        local_datetime = gocprefix[1]
        print('Found local cache from: ',local_datestamp)
    else:
        print('Error no local cache of '+ gocprefixfile + ' available')
        local_datetime = None
        
print(prefix in gocprefix) 
interval = time.time() - then
print('checked ' + str(len(gocprefix)) + ' prefixes from GO for ' + prefix + ' in ' + str(interval))

False
checked 313 prefixes from GO for BUNNYFOOFOO in 0.907667875289917


In [5]:
"GO" in gocprefix

True

In [6]:
for p in gocprefix:
    if re.match(r'.*/.*', p):
        print(p)

shows these GOC prefixes are __not__ composed  
of namespaces preceded 
by an optional "provider code" then slash

___

## CDL/EBI

In [7]:
cdlebi_url = 'https://n2t.net/e/cdl_ebi_prefixes.yaml'
response = requests.get(cdlebi_url)

In [8]:
first_line = response.text[0:response.text.index('\n')]
print(first_line)
# Last modified: 2017.08.01_08.28.20
fmt = '# Last modified: %Y.%m.%d_%H.%M.%S'
cdlebi_file = 'cdl_ebi_prefixes.yaml'
cdlebipfx = []

# Last modified: 2017.08.01_08.28.20


In [9]:
then = time.time()
if response.status_code == requests.codes.ok:
    cdlebiraw = response.text
    cdlebireg = yaml.load(cdlebiraw)
    first_line = cdlebiraw[0:cdlebiraw.index('\n')]
    remote_datetime = datetime.strptime(first_line, fmt) 
else: 
    print('ERROR ' + response.url + ' returned '+ str(response.status_code))
    print('Trying local cache')
    remote_datetime = None
    if os.path.isfile(cdlebi_file):
        with open(cdlebi_file, 'r') as fh:
            cdlebireg = yaml.load(fh)
            # local_datetime = cdlebireg ... not there would to read need the raw file
            # print('Found local cache from: ',local_datestamp)
    else:
        print('Error no local cache of '+ cdlebi_file + ' available')
        local_datetime = None

for reg in cdlebireg:
    cdlebipfx.append(reg['namespace'])
    if 'alias' in reg:
        cdlebipfx.append(reg['alias'])
        
print(prefix in cdlebipfx)
interval = time.time() - then
print('checked ' + str(len(cdlebipfx)) + ' prefixes from CDL/EBI for ' + prefix + ' in ' + str(interval))

False
checked 662 prefixes from CDL/EBI for BUNNYFOOFOO in 0.4575526714324951



a second or less for both of  the yaml fetch checks   
(much less on average if we keep the lists in server memory and only fetch to update periodicaly/when changed)  

several to ~10 seconds to ping the remote servers if a hit is not found yet


In [10]:
len(cdlebireg)

652